Name: Patrick Ng  
Email: patng@ischool.berkeley.edu  
Class: W261-2  
Week: 03  
Date of submission: Feb 01, 2016

## HW3.0.

What is a merge sort? Where is it used in Hadoop?  
How is  a combiner function in the context of Hadoop?   
Give an example where it can be used and justify why it should be used in the context of this problem.  
What is the Hadoop shuffle?

TBD

## HW3.1 
Use Counters to do EDA (exploratory data analysis and to monitor progress)
Counters are lightweight objects in Hadoop that allow you to keep track of system progress in both the map and reduce stages of processing. By default, Hadoop defines a number of standard counters in "groups"; these show up in the jobtracker webapp, giving you information such as "Map input records", "Map output records", etc. 

While processing information/data using MapReduce job, it is a challenge to monitor the progress of parallel threads running across nodes of distributed clusters. Moreover, it is also complicated to distinguish between the data that has been processed and the data which is yet to be processed. The MapReduce Framework offers a provision of user-defined Counters, which can be effectively utilized to monitor the progress of data across nodes of distributed clusters.

Use the Consumer Complaints  Dataset provide here to complete this question:

     https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0

The consumer complaints dataset consists of diverse consumer complaints, which have been reported across the United States regarding various types of loans. The dataset consists of records of the form:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?


User-defined Counters

Now, let’s use Hadoop Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories (all other categories get lumped into this one) in the consumer complaints dataset. Basically produce the distribution of the Product column in this dataset using counters (limited to 3 counters here).

Hadoop offers Job Tracker, an UI tool to determine the status and statistics of all jobs. Using the job tracker UI, developers can view the Counters that have been created. Screenshot your  job tracker UI as your job completes and include it here. Make sure that your user defined counters are visible. 

## Mapper

In [73]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

# input comes from STDIN (standard input)
for line in sys.stdin:
    fields = line.split(",")

    # Skip header row
    if fields[0] == "Complaint ID":
        continue

    reason = fields[1].lower()
    
    if reason == "debt collection":
        counter = "debt"
    elif reason == "mortgage":
        counter = "mortgage"
    else:
        counter = "others"
        
    sys.stderr.write('reporter:counter:custom,' + counter + ',1\n')


Overwriting mapper.py


## Quick Test

In [74]:
# Quick test
!head -n 20 Consumer_Complaints.csv  | python mapper.py

reporter:counter:custom,debt,1
reporter:counter:custom,debt,1
reporter:counter:custom,others,1
reporter:counter:custom,debt,1
reporter:counter:custom,others,1
reporter:counter:custom,debt,1
reporter:counter:custom,others,1
reporter:counter:custom,debt,1
reporter:counter:custom,others,1
reporter:counter:custom,debt,1
reporter:counter:custom,others,1
reporter:counter:custom,others,1
reporter:counter:custom,debt,1
reporter:counter:custom,mortgage,1
reporter:counter:custom,others,1
reporter:counter:custom,others,1
reporter:counter:custom,debt,1
reporter:counter:custom,debt,1
reporter:counter:custom,debt,1


## Run it in Hadoop

In [2]:
# upload input file to hdfs
!hdfs dfs -rm -f Consumer_Complaints.csv
!hdfs dfs -put Consumer_Complaints.csv

16/01/30 17:26:52 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/30 17:26:54 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [16]:
# Hadoop streaming command
!hdfs dfs -rm -r hw3.1
!hadoop jar $HADOOP_INSTALL/share/hadoop/tools/lib/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=0 \
-file mapper.py -mapper mapper.py \
-input Consumer_Complaints.csv \
-output hw3.1

16/01/30 17:40:10 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted hw3.1
16/01/30 17:40:12 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/30 17:40:12 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [mapper.py, /var/folders/dm/nsw7wjf91f1c74hgl17ldw040000gn/T/hadoop-unjar6298878087895734314/] [] /var/folders/dm/nsw7wjf91f1c74hgl17ldw040000gn/T/streamjob2623231940027838210.jar tmpDir=null


### Result

![result](https://photos-3.dropbox.com/t/2/AAD96apEVb1NEyOodPWCtkdcs8K_w-nW4PJtqZe6LTUSCQ/12/15674996/png/32x32/1/_/1/2/Screenshot%202016-01-30%2017.42.35.png/EKi01gsYnS8gBygH/Hs_dhV-YD5vL1Ja5tyxj1tFInEhRb2H_uNNtVF96zBs?size=1024x768&size_mode=3)

## HW 3.2  - Part 1

Analyze the performance of your Mappers, Combiners and Reducers using Counters

For this brief study the Input file will be one record (the next line only): 
foo foo quux labs foo bar quux


Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain.

Please use mulitple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 
Using a single reducer: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

In [17]:
# Prepare input file
!echo "foo foo quux labs foo bar quux" > input.txt
!hdfs dfs -rm -r input.txt
!hdfs dfs -put input.txt

16/01/30 17:53:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `input.txt': No such file or directory
16/01/30 17:53:38 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Mapper and Reducer

In [163]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

# input comes from STDIN (standard input)
for line in sys.stdin:
    sys.stderr.write('reporter:counter:custom,mapper_called,1\n')
    words = line.strip().split(" ")
    for word in words:
        print word + "\t1" 
            

Overwriting mapper.py


In [166]:
%%writefile reducer.py
#!/usr/bin/python
import sys

wordCount = 0
prev = None # the word previously seen

# input comes from STDIN
for line in sys.stdin:
    (word, count) = line.strip().split('\t')
    count = int(count)
    
    # If we have encountered a new word, output the answer of the previous word
    if prev != word:
        if prev is not None:
            sys.stderr.write('reporter:counter:custom,reducer_called,1\n')
            print "%s\t%d" % (prev, wordCount)
            wordCount = 0
            
    wordCount += count
    prev = word


# Output for the last word seen
if prev is not None:
    sys.stderr.write('reporter:counter:custom,reducer_called,1\n')
    print "%s\t%d" % (prev, wordCount)

Overwriting reducer.py


### Quick Test

In [167]:
!cat input.txt | python mapper.py | sort | python reducer.py

reporter:counter:custom,mapper_called,1
reporter:counter:custom,reducer_called,1
bar	1
reporter:counter:custom,reducer_called,1
foo	3
reporter:counter:custom,reducer_called,1
labs	1
reporter:counter:custom,reducer_called,1
quux	2


### Run in Hadoop

In [168]:
!hdfs dfs -rm -r hw3.2-part1

!hadoop jar $HADOOP_INSTALL/share/hadoop/tools/lib/hadoop-*streaming*.jar \
-file mapper.py -mapper mapper.py \
-file reducer.py -reducer reducer.py \
-input input.txt \
-output hw3.2-part1

16/01/31 18:22:50 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted hw3.2-part1
16/01/31 18:22:52 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/31 18:22:52 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [mapper.py, reducer.py, /var/folders/dm/nsw7wjf91f1c74hgl17ldw040000gn/T/hadoop-unjar1822858052759819100/] [] /var/folders/dm/nsw7wjf91f1c74hgl17ldw040000gn/T/streamjob7516553153088005811.jar tmpDir=null


In [169]:
# Show result
!hdfs dfs -cat hw3.2-part1/part-00000

16/01/31 18:23:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
bar	1
foo	3
labs	1
quux	2


### Result Part 1

The mapper was called once, because there is only one line of input.  
The reducer was called four times, because there are four words.


## HW 3.2  - Part 2

Please use mulitple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

### Mapper (reduce last Reducer)

In [170]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

# Word delimiters are space, \ and "
regex = re.compile(r"[\s/\"]+")

# input comes from STDIN (standard input)
for line in sys.stdin:
    fields = line.strip().split(",")

    # Skip header row
    if fields[0] == "Complaint ID":
        continue

    sys.stderr.write('reporter:counter:custom,mapper_called,1\n')
    
    words = filter(None, regex.split(fields[3]))
    for word in words:
        print word + "\t1" 
            

Overwriting mapper.py


### Quick Test

In [171]:
!head -n 4 Consumer_Complaints.csv | python mapper.py | sort | python reducer.py

reporter:counter:custom,mapper_called,1
reporter:counter:custom,mapper_called,1
reporter:counter:custom,mapper_called,1
reporter:counter:custom,reducer_called,1
Deposits	1
reporter:counter:custom,reducer_called,1
Disclosure	2
reporter:counter:custom,reducer_called,1
and	1
reporter:counter:custom,reducer_called,1
debt	2
reporter:counter:custom,reducer_called,1
of	2
reporter:counter:custom,reducer_called,1
verification	2
reporter:counter:custom,reducer_called,1
withdrawals	1


### Run in Hadoop

In [172]:
!hdfs dfs -rm -r hw3.2-part2

# Use 4 mappers and 4 reducers
!hadoop jar $HADOOP_INSTALL/share/hadoop/tools/lib/hadoop-*streaming*.jar \
-D mapred.map.tasks=4 -D mapred.reduce.tasks=4 \
-file mapper.py -mapper mapper.py \
-file reducer.py -reducer reducer.py \
-input Consumer_Complaints.csv \
-output hw3.2-part2

16/01/31 18:23:52 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted hw3.2-part2
16/01/31 18:23:54 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/31 18:23:54 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [mapper.py, reducer.py, /var/folders/dm/nsw7wjf91f1c74hgl17ldw040000gn/T/hadoop-unjar6488566181085939433/] [] /var/folders/dm/nsw7wjf91f1c74hgl17ldw040000gn/T/streamjob1722145701917745206.jar tmpDir=null


### Result Part 2

The mapper was called 312,912 times.  
The reducer was called 180 times.

## HW3.2 - Part 3

Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job.

### Combiner (Note: Re-use the previous mapper and reducer)

In [173]:
%%writefile combiner.py
#!/usr/bin/python
import sys

wordCount = 0
prev = None # the word previously seen

# input comes from STDIN
for line in sys.stdin:
    (word, count) = line.split('\t')
    count = int(count)
    
    # If we have encountered a new word, output the answer of the previous word
    if prev != word:
        if prev is not None:
            sys.stderr.write('reporter:counter:custom,combiner_called,1\n')
            print "%s\t%d" % (prev, wordCount)
            wordCount = 0
            
    wordCount += count
    prev = word


# Output for the last word seen
if prev is not None:
    sys.stderr.write('reporter:counter:custom,combiner_called,1\n')
    print "%s\t%d" % (prev, wordCount)

Overwriting combiner.py


### Quick Test

In [174]:
!head -n 4 Consumer_Complaints.csv | python mapper.py | python combiner.py | sort | python reducer.py

reporter:counter:custom,mapper_called,1
reporter:counter:custom,mapper_called,1
reporter:counter:custom,mapper_called,1
reporter:counter:custom,combiner_called,1
reporter:counter:custom,combiner_called,1
reporter:counter:custom,combiner_called,1
reporter:counter:custom,combiner_called,1
reporter:counter:custom,combiner_called,1
reporter:counter:custom,combiner_called,1
reporter:counter:custom,combiner_called,1
reporter:counter:custom,combiner_called,1
reporter:counter:custom,combiner_called,1
reporter:counter:custom,combiner_called,1
reporter:counter:custom,combiner_called,1
reporter:counter:custom,reducer_called,1
Deposits	1
reporter:counter:custom,reducer_called,1
Disclosure	2
reporter:counter:custom,reducer_called,1
and	1
reporter:counter:custom,reducer_called,1
debt	2
reporter:counter:custom,reducer_called,1
of	2
reporter:counter:custom,reducer_called,1
verification	2
reporter:counter:custom,reducer_called,1
withdrawals	1


In [175]:
!hdfs dfs -rm -r hw3.2-part3

# Use 4 mappers and 4 reducers
!hadoop jar $HADOOP_INSTALL/share/hadoop/tools/lib/hadoop-*streaming*.jar \
-D mapred.map.tasks=4 -D mapred.reduce.tasks=4 \
-file mapper.py -mapper mapper.py \
-file combiner.py -combiner combiner.py \
-file reducer.py -reducer reducer.py \
-input Consumer_Complaints.csv \
-output hw3.2-part3

16/01/31 18:25:25 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted hw3.2-part3
16/01/31 18:25:27 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/31 18:25:27 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [mapper.py, combiner.py, reducer.py, /var/folders/dm/nsw7wjf91f1c74hgl17ldw040000gn/T/hadoop-unjar5822869549960484937/] [] /var/folders/dm/nsw7wjf91f1c74hgl17ldw040000gn/T/streamjob1339676925685870465.jar tmpDir=null


### Result Part 3

The mapper was called 312,912 times.  
The combiner was called 628 times.  
The reducer was called 180 times.

## HW 3.2 Part 4

Using a single reducer: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

### Mapper and Reducer

In [194]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

# input comes from STDIN (standard input)
for line in sys.stdin:
    (word, count) = line.strip().split("\t")

    sys.stderr.write('reporter:counter:custom,mapper_called,1\n')
    
    print count + "\t" + word 

    # Use order inversion so that reducer can count the total word count in a single pass
    print str(sys.maxint) + "\t" + count

Overwriting mapper.py


In [220]:
%%writefile reducer.py
#!/usr/bin/python
from __future__ import division # Use Python 3-style division
import sys, Queue

n_max, n_min = 50, 10
q_min = Queue.Queue(n_min)
a_max = []

def updateResult(word, count, freq):
    global n_max, q_min, a_max
    
    rec = [word, count, freq]
    # put the biggest
    if len(a_max) < n_max:
        a_max.append(rec)
    
    # whatever left is the smallest
    if q_min.full():
        q_min.get()
    q_min.put(rec)
    
wordCount = 0 # Count of each word
totalCount = 0 # Total number of words
prev = None # the word previously seen

# input comes from STDIN
for line in sys.stdin:
    (count, word) = line.strip().split('\t')
    count = int(count)

    # Find out the total word count.
    # We use count == sys.maxint as the special key for order inversion.
    if count == sys.maxint:
        totalCount += int(word) # The word is the count
        continue
    
    # If we have encountered a new word, output the answer of the previous word
    if prev != word:
        if prev is not None:
            sys.stderr.write('reporter:counter:custom,reducer_called,1\n')
            updateResult(prev, wordCount, wordCount/totalCount)
            wordCount = 0
            
    wordCount += count
    prev = word

# Handle the last word seen
if prev is not None:
    sys.stderr.write('reporter:counter:custom,reducer_called,1\n')
    updateResult(prev, wordCount, wordCount/totalCount)
    
# Output the result
print "Top %d words" % n_max
for rec in a_max:
    print "%s\t%d\t%f" % tuple(rec)
    
print
print "Bottom %d words" % n_min
while not q_min.empty():
    print "%s\t%d\t%f" % tuple(q_min.get())


Overwriting reducer.py


### Quick test

In [212]:
# Prepare test input
!hdfs dfs -getmerge hw3.2-part3/part-* hw3.2-part4-input.txt
!hdfs dfs -put hw3.2-part4-input.txt

16/01/31 19:33:13 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 19:33:15 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
put: `hw3.2-part4-input.txt': File exists


In [221]:
!head -n 4 hw3.2-part4-input.txt | python mapper.py | sort -g -t\t -k1 -r | python reducer.py

reporter:counter:custom,mapper_called,1
reporter:counter:custom,mapper_called,1
reporter:counter:custom,mapper_called,1
reporter:counter:custom,mapper_called,1
reporter:counter:custom,reducer_called,1
reporter:counter:custom,reducer_called,1
reporter:counter:custom,reducer_called,1
reporter:counter:custom,reducer_called,1
Top 50 words
Account	16555	0.874399
Can't	1999	0.105583
Cash	240	0.012676
Applied	139	0.007342

Bottom 10 words
Account	16555	0.874399
Can't	1999	0.105583
Cash	240	0.012676
Applied	139	0.007342


### Run it in Hadoop

In [222]:
!hdfs dfs -rm -r hw3.2-part4

# For sorting, use the 1st field (count) as the primary key, in reverse order.
# Since we specify 2 key fields, the 2nd field (word) will be used as secondary key.
!hadoop jar $HADOOP_INSTALL/share/hadoop/tools/lib/hadoop-*streaming*.jar \
-D mapred.reduce.tasks=1 \
-D stream.num.map.output.key.fields=2 \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapred.text.key.comparator.options="-k1nr -k2" \
-file mapper.py -mapper mapper.py \
-file reducer.py -reducer reducer.py \
-input hw3.2-part3/part-* \
-output hw3.2-part4


16/01/31 19:45:40 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Deleted hw3.2-part4
16/01/31 19:45:42 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/31 19:45:42 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [mapper.py, reducer.py, /var/folders/dm/nsw7wjf91f1c74hgl17ldw040000gn/T/hadoop-unjar452543312068260830/] [] /var/folders/dm/nsw7wjf91f1c74hgl17ldw040000gn/T/streamjob5904996276397581685.jar tmpDir=null


### Result for Part 4

In [223]:
!hdfs dfs -cat hw3.2-part4/part-00000

16/01/31 19:46:16 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Top 50 words	
Loan	107254	0.109389
modification	70487	0.071890
credit	40483	0.041289
servicing	36767	0.037499
report	34903	0.035598
Incorrect	29133	0.029713
information	29069	0.029648
on	29069	0.029648
or	22533	0.022982
debt	17966	0.018324
Account	16555	0.016885
and	16448	0.016775
opening	16205	0.016528
Credit	14768	0.015062
club	12545	0.012795
health	12545	0.012795
loan	12376	0.012622
not	12353	0.012599
Cont'd	11848	0.012084
attempts	11848	0.012084
collect	11848	0.012084
owed	11848	0.012084
of	10885	0.011102
my	10731	0.010945
Deposits	10555	0.010765
withdrawals	10555	0.010765
Problems	9484	0.009673
Application	8868	0.009045
to	8401	0.008568
Billing	8158	0.008320
Other	7886	0.008043
disputes	6938	0.007076
Communication	6920	0.007058
tactics	6920	0.007058
reporting	6559	0.006690
lease	6337	0.006463
the	6248	0.006372
being	5663	0.005776
by	56

## HW3.2.1 OPTIONAL 
Using 2 reducers: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

In [ ]:
?????

## HW3.3. 

Shopping Cart Analysis  
Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.  

For this homework use the online browsing behavior dataset located at: 

       https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

Each line in this dataset represents a browsing session of a customer. 
On each line, each string of 8 characters represents the id of an item browsed during that session.   
The items are separated by spaces.


Do some exploratory data analysis of this dataset. 

How many unique items are available from this supplier?

Using a single reducer: Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 

In [ ]:
????

In [194]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

# input comes from STDIN (standard input)
for line in sys.stdin:
    items = line.strip().split()
    
    print count + "\t" + word 

    # Use order inversion so that reducer can count the total word count in a single pass
    print str(sys.maxint) + "\t" + count

Overwriting mapper.py


## 3.3.1 OPTIONAL 
Using 2 reducers:  Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 

## HW3.4. Pairs

Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a map-reduce program 
to find products which are frequently browsed together. Fix the support count (cooccurence count) to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) that have a support count of 100 or more.

List the top 50 product pairs with corresponding support count (aka frequency), and relative frequency or support (the number of records where they coccur/the number of baskets in the dataset) in decreasing order of support  for frequent (100>count) itemsets of size 2. 

Use the Pairs pattern (lecture 3) to  extract these frequent itemsets of size 2. Free free to use combiners if they bring value. Instrument your code with counters for count the number of times your mapper, combiner and reducers are called.  

Please output records of the following form for the top 50 pairs (itemsets of size 2): 

      item1, item2, support count, support



Fix the ordering of the pairs lexicographically (left to right), 
and break ties in support (between pairs, if any exist) 
by taking the first ones in lexicographically increasing order. 

Report  the compute time for the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)
Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts.

### Mapper and Reducer 

In [282]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

# input comes from STDIN (standard input)
for line in sys.stdin:
    items = line.strip().split()
    
    sys.stderr.write('reporter:counter:custom,mapper_called,1\n')

    # Use order inversion to help calculate the total # of baskets
    print "#\t\t1" # Use # as the special key
    
    items = list(set(items)) # Remove duplicate items

    # sort the items as we need to fix the ordering of the pairs lexicographically
    items = sorted(items)
    
    # First ignore the corner case
    if len(items) <= 1:
        continue
        
    # Output all co-occuring pairs.   
    for i in range(len(items) - 1):
        x = items[i]
        for j in range(i+1, len(items)):
            y = items[j]
            print "%s\t%s\t1" % (x, y)
            

Overwriting mapper.py


In [283]:
%%writefile reducer.py
#!/usr/bin/python
from __future__ import division # Use Python 3-style division
import sys

supportCount = 0 # Support count of each pair
totalCount = 0 # Total number of baskets
lastPair = None # The pair previously seen
supportThreshold = 100

top50 = []

# Maintain the top50 list
def updateResults(item1, item2, supportCount, support):
    global top50
    
    top50.append([item1, item2, supportCount, support])
    
    # Sort by count in descending order, then by the word in ascending order.
    # The trick here is to use "negative count" as the first key to get the descending order
    # but keep the ascending order in the other keys
    top50 = sorted(top50, key = lambda x: (-1 * x[2], x[0], x[1]))
    
    # Remove the last one if the list is full
    if len(top50) > 50:
        top50.pop(50)
        
        
# input comes from STDIN
for line in sys.stdin:
    (item1, item2, count) = line.strip().split('\t')
    count = int(count)

    # Find out the total basket count.
    # We use # as the special key for order inversion.
    if item1 == "#":
        totalCount += count
        continue
        
    pair = [item1, item2]
    
    # If we have encountered a new pair, output the count of the last pair
    if lastPair and lastPair != pair:
        sys.stderr.write('reporter:counter:custom,reducer_called,1\n')
        if supportCount >= supportThreshold:
            updateResults(lastPair[0], lastPair[1], supportCount, supportCount / totalCount)
        supportCount = 0
            
    supportCount += count
    lastPair = pair

# Handle the last pair seen
if lastPair is not None:
    sys.stderr.write('reporter:counter:custom,reducer_called,1\n')
    if supportCount >= supportThreshold:
        updateResults(lastPair[0], lastPair[1], supportCount, supportCount / totalCount)
    

# Output result
for rec in top50:
    print "%s\t%s\t%d\t%f" % (rec[0], rec[1], rec[2], rec[3])

Overwriting reducer.py


### Quick test

In [268]:
!head -n 2 ProductPurchaseData.txt | \
python mapper.py 2>/dev/null | \
sort -k1 | python reducer.py 2>/dev/null 

Top 50 pairs
ELE17451, GRO99222, 2, 1.000000
ELE17451, ELE26917, 1, 0.500000
ELE17451, ELE52966, 1, 0.500000
ELE17451, ELE89019, 1, 0.500000
ELE17451, ELE91550, 1, 0.500000
ELE17451, FRO11987, 1, 0.500000
ELE17451, FRO12685, 1, 0.500000
ELE17451, FRO84225, 1, 0.500000
ELE17451, FRO90334, 1, 0.500000
ELE17451, GRO12298, 1, 0.500000
ELE17451, SNA11465, 1, 0.500000
ELE17451, SNA30755, 1, 0.500000
ELE17451, SNA80192, 1, 0.500000
ELE17451, SNA90258, 1, 0.500000
ELE26917, ELE52966, 1, 0.500000
ELE26917, ELE91550, 1, 0.500000
ELE26917, FRO12685, 1, 0.500000
ELE26917, FRO84225, 1, 0.500000
ELE26917, FRO90334, 1, 0.500000
ELE26917, GRO12298, 1, 0.500000
ELE26917, GRO99222, 1, 0.500000
ELE26917, SNA11465, 1, 0.500000
ELE26917, SNA30755, 1, 0.500000
ELE26917, SNA80192, 1, 0.500000
ELE52966, ELE91550, 1, 0.500000
ELE52966, FRO12685, 1, 0.500000
ELE52966, FRO84225, 1, 0.500000
ELE52966, FRO90334, 1, 0.500000
ELE52966, GRO12298, 1, 0.500000
ELE52966, GRO99222, 1, 0.500000
ELE52966, SNA11465, 1, 0.50

### Run it in Hadoop

In [269]:
# upload input file to hdfs
!hdfs dfs -rm -f ProductPurchaseData.txt
!hdfs dfs -put ProductPurchaseData.txt

16/01/31 23:09:40 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 23:09:42 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [284]:
!hdfs dfs -rm -r hw3.4

!hadoop jar $HADOOP_INSTALL/share/hadoop/tools/lib/hadoop-*streaming*.jar \
-D mapred.map.tasks=3 \
-D mapred.reduce.tasks=1 \
-D stream.num.map.output.key.fields=2 \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapred.text.key.comparator.options="-k1 -k2" \
-file mapper.py -mapper mapper.py \
-file reducer.py -reducer reducer.py \
-input ProductPurchaseData.txt \
-output hw3.4

16/01/31 23:59:03 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `hw3.4': No such file or directory
16/01/31 23:59:05 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/01/31 23:59:05 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [mapper.py, reducer.py, /var/folders/dm/nsw7wjf91f1c74hgl17ldw040000gn/T/hadoop-unjar3348896373397896635/] [] /var/folders/dm/nsw7wjf91f1c74hgl17ldw040000gn/T/streamjob4287849993728212145.jar tmpDir=null


In [285]:
# Show the result
!hdfs dfs -cat hw3.4/part-00000

16/01/31 23:59:57 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
DAI62779	ELE17451	1592	0.051188
FRO40251	SNA80324	1412	0.045400
DAI75645	FRO40251	1254	0.040320
FRO40251	GRO85051	1213	0.039002
DAI62779	GRO73461	1139	0.036623
DAI75645	SNA80324	1130	0.036333
DAI62779	FRO40251	1070	0.034404
DAI62779	SNA80324	923	0.029678
DAI62779	DAI85309	918	0.029517
ELE32164	GRO59710	911	0.029292
DAI62779	DAI75645	882	0.028359
FRO40251	GRO73461	882	0.028359
DAI62779	ELE92920	877	0.028198
FRO40251	FRO92469	835	0.026848
DAI62779	ELE32164	832	0.026752
DAI75645	GRO73461	712	0.022893
DAI43223	ELE32164	711	0.022861
DAI62779	GRO30386	709	0.022797
ELE17451	FRO40251	697	0.022411
DAI85309	ELE99737	659	0.021189
DAI62779	ELE26917	650	0.020900
GRO21487	GRO73461	631	0.020289
DAI62779	SNA45677	604	0.019421
ELE17451	SNA80324	597	0.019196
DAI62779	GRO71621	595	0.019131
DAI62779	SNA55762	593	0.019067
DAI62779	DAI83733	586	0.018842
ELE17451

### Report 

Compute time: 38sec   
Setup: Quad Core, OS X, 3 mappers, 1 reducer  
Mapper called: 31,101 times  
Reducer called: 877,095 times  